In [1]:
import pandas as pd

books = pd.read_csv('books_with_categories.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'books_with_categories.csv'

In [2]:
from transformers import pipeline

classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None, device="mps")
classifier("I love this")

c:\Users\LENOVO\Desktop\BookRecommendatioin\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\LENOVO\Desktop\BookRecommendatioin\venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\LENOVO\.cache\huggingface\hub\models--j-hartmann--emotion-english-distilroberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Py

[[{'label': 'anger', 'score': 0.0013875303557142615},
  {'label': 'disgust', 'score': 0.0007134026964195073},
  {'label': 'fear', 'score': 0.0003984913637395948},
  {'label': 'joy', 'score': 0.9845667481422424},
  {'label': 'neutral', 'score': 0.0034752979408949614},
  {'label': 'sadness', 'score': 0.004531427752226591},
  {'label': 'surprise', 'score': 0.00492720166221261}]]

In [ ]:
books['description'][0]

In [ ]:
classifier(books["description"][0])

In [ ]:
# Splitting the description into sentences and applying 
# classification on it. 

classifier(books["description"][0].split("."))


In [ ]:
sentences = books["description"][0].split(".")
predictions = classifier(sentences)

In [ ]:
sentences[0]

In [ ]:
predictions[0]

In [ ]:
# sorting by score 
sorted(predictions[0], key=lambda x: x["label"])

In [ ]:
# Getting the maximum emotion probability
import numpy as np 

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}

def calculate_max_emotion_score(predictions):
    per_emotion_scores = {label: [] for label in emotion_labels}
    for prediction in predictions:
        sorted_predictions = sorted(prediction, key=lambda x: x["label"])
        for index, label in enumerate(emotion_labels):
            per_emotion_scores[label].append(sorted_predictions[index]["score"])
    
    return {label: np.max(scores) for label, scores in per_emotion_scores.item()}


In [ ]:
for i in range(10):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_score(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])




In [ ]:
emotion_scores

In [ ]:
from tqdm import tqdm

emotion_labels = ["anger", "disgust", "fear", "joy", "sadness", "surprise", "neutral"]
isbn = []
emotion_scores = {label: [] for label in emotion_labels}


for i in tqdm(range(len(books))):
    isbn.append(books["isbn13"][i])
    sentences = books["description"][i].split(".")
    predictions = classifier(sentences)
    max_scores = calculate_max_emotion_score(predictions)
    for label in emotion_labels:
        emotion_scores[label].append(max_scores[label])




In [ ]:
emotions_df = pd.DataFrame(emotion_scores)
emotions_df["isbn13"] = isbn
emotions_df.head(2)

In [ ]:
books = pd.merge(books, emotions_df, on="isbn13")

In [ ]:
books.to_csv("books_with_emotions.csv", index=False)